## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-07-21-53-01 +0000,bbc,"Benin coup thwarted by loyalist troops, presid...",https://www.bbc.com/news/articles/c62v7n9wzkyo...
1,2025-12-07-21-46-24 +0000,nypost,Holiday parade in California’s wealthy ‘Americ...,https://nypost.com/2025/12/07/us-news/holiday-...
2,2025-12-07-21-44-45 +0000,cbc,Conservative motion will force Liberals to 'pu...,https://www.cbc.ca/news/politics/poilievre-con...
3,2025-12-07-21-44-39 +0000,nypost,Dirty-minded Long Island residents outspend en...,https://nypost.com/2025/12/07/us-news/long-isl...
4,2025-12-07-21-41-56 +0000,nyt,Netflix Deal for Warner Bros. Adds New Twist t...,https://www.nytimes.com/2025/12/07/business/me...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2344/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
172,trump,22
170,fire,11
41,new,11
21,will,7
128,out,7


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
78,2025-12-07-16-27-00 +0000,wsj,President Trump criticized as disloyal an indi...,https://www.wsj.com/politics/elections/trump-l...,51
30,2025-12-07-20-38-10 +0000,nypost,"Trump takes over Washington’s most glam night,...",https://nypost.com/2025/12/07/us-news/trump-ta...,49
79,2025-12-07-16-24-42 +0000,nypost,Tim Walz’s daughter Hope goes on foul-mouthed ...,https://nypost.com/2025/12/07/us-news/tim-walz...,47
44,2025-12-07-19-42-14 +0000,cbc,Trump's new security strategy aligns with Russ...,https://www.cbc.ca/news/world/russia-trump-nat...,46
91,2025-12-07-13-42-20 +0000,cbc,"Israel's Netanyahu says he will meet Trump, se...",https://www.cbc.ca/news/world/netanyahu-says-h...,46


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
78,51,2025-12-07-16-27-00 +0000,wsj,President Trump criticized as disloyal an indi...,https://www.wsj.com/politics/elections/trump-l...
168,43,2025-12-06-23-09-00 +0000,wsj,Defense Secretary Pete Hegseth defended the U....,https://www.wsj.com/politics/national-security...
101,37,2025-12-07-12-33-07 +0000,nypost,Hochul will drive away New Yorkers if she supp...,https://nypost.com/2025/12/07/us-news/pataki-h...
51,35,2025-12-07-19-11-36 +0000,startribune,A fire warning for an e-bike battery was issue...,https://www.startribune.com/a-fire-warning-for...
37,34,2025-12-07-20-13-28 +0000,nypost,Tom Homan defends ICE’s crackdown on Somali mi...,https://nypost.com/2025/12/07/us-news/tom-homa...
12,25,2025-12-07-21-28-28 +0000,nyt,Presidential Power Gets Tested Before Supreme ...,https://www.nytimes.com/2025/12/07/us/politics...
18,25,2025-12-07-21-16-54 +0000,nypost,Female customers at ritzy NYC eatery report bl...,https://nypost.com/2025/12/07/us-news/female-c...
79,25,2025-12-07-16-24-42 +0000,nypost,Tim Walz’s daughter Hope goes on foul-mouthed ...,https://nypost.com/2025/12/07/us-news/tim-walz...
1,24,2025-12-07-21-46-24 +0000,nypost,Holiday parade in California’s wealthy ‘Americ...,https://nypost.com/2025/12/07/us-news/holiday-...
15,24,2025-12-07-21-23-01 +0000,nypost,Hawaii’s Kīlauea volcano destroys camera captu...,https://nypost.com/2025/12/07/us-news/hawaiis-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
